#CafChem Teaching - A chatbot using OpenAI's open weights model, GPT-OSS-20B.

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/MauricioCafiero/CafChemTeach/blob/main/notebooks/OpenAI_Chatbot_CafChem.ipynb)

## This notebook allows you to:
- Download the weights for the OpenAI GPT-OSS-20B model.
- Interact with the model via a chatbot.

## Requirements:
- If using on Colab, it will (un)install all needed libraries.
- Access to your HF token
- Needs a GPU, L4 or higher preferably (will be quite slow on T4).

## Set-up

The OpenAI open models have quite complicated dependencies, and so several packages have to be install/un-installed

In [ ]:
 !pip install -q --upgrade torch


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 888.1/888.1 MB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 594.3/594.3 MB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.2/10.2 MB 123.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.0/88.0 MB 25.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 954.8/954.8 kB 54.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 706.8/706.8 MB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.1/193.1 MB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 60.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.6/63.6 MB 35.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 267.5/267.5 MB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 288.2/288.2 MB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 287.2/287.2 MB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 

In [ ]:
!pip install -q git+https://github.com/huggingface/transformers triton==3.4 git+https://github.com/triton-lang/triton.git@main#subdirectory=python/triton_kernels

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


In [ ]:
!pip uninstall -q torchvision torchaudio -y

In [ ]:
!pip install bitsandbytes

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.9/72.9 MB 32.6 MB/s eta 0:00:00


In [ ]:
pip install kernels

In [ ]:
import os
import re
import transformers
import torch
import gradio as gr
from transformers import AutoModelForCausalLM, AutoTokenizer
from transformers import Mxfp4Config
import triton
import kernels

## Functions
- These functions set-up the model and power the chatbot

In [ ]:
chat_history = []
global last_id
last_id = None

def clear_history():
        global chat_history
        chat_history = []
        global last_id
        last_id = None

def setup_free_openai():
  '''
  '''
  model_id = "openai/gpt-oss-20b"
  tokenizer = AutoTokenizer.from_pretrained(model_id)

  quantization_config = Mxfp4Config(dequantize=False)

  model = AutoModelForCausalLM.from_pretrained(
          model_id,
          device_map="cuda",
          torch_dtype="auto",
          quantization_config = quantization_config)

  return tokenizer, model

def chat(prompt, reason):
  '''
  '''
  global last_id
  global chat_history
  chat_history.append({"role": "user", "content": prompt, "reasoning_effort": reason})

  inputs = tokenizer.apply_chat_template(
                      chat_history, #messages,
                      add_generation_prompt=True,
                      return_tensors="pt",
                      return_dict=True).to(model.device)

  generated = model.generate(**inputs, max_new_tokens=1000)
  response = tokenizer.decode(generated[0][inputs["input_ids"].shape[-1]:])

  start_match = "\<\|message\|\>"
  response_parts = re.split(start_match, response)
  answer = response_parts[-1].replace('<|return|>','')
  reasoning = response_parts[-2].replace('<|return|>','')

  chat_history.append(
              {"role": "assistant", "content": answer, "reasoning": reasoning}
  )
  print(answer)


  return "", chat_history, reasoning


def chatbot():
  '''
  '''
  with gr.Blocks() as forest:
    gr.Markdown(
        """
        # Chat with OpenAI GPT-OSS-20B.
        - If using high reasoning, responses can take a minute or longer!
        ### Enter your messages below.
        """)

    reason = gr.Radio(choices = ["low", "medium", "high"],label="Reasoning Level",
                    value = "medium", interactive=True)


    chatbot = gr.Chatbot(type="messages")
    msg = gr.Textbox(label="Type your messages here and hit enter.")


    reasoning_output = gr.Textbox(label="Reasoning", value="")

    chat_btn = gr.Button(value = "Send")

    clear = gr.ClearButton([msg, chatbot, reasoning_output])
    clear.click(clear_history)


    chat_btn.click(chat, [msg, reason], [msg, chatbot, reasoning_output])
    msg.submit(chat, [msg, reason], [msg, chatbot, reasoning_output])


  forest.launch(share=True)

## Download the model weights and tokenizer from HuggingFace

In [ ]:
tokenizer, model = setup_free_openai()

tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.json:   0%|          | 0.00/27.9M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/98.0 [00:00<?, ?B/s]

chat_template.jinja: 0.00B [00:00, ?B/s]

config.json: 0.00B [00:00, ?B/s]

...

blackwell_scale.py: 0.00B [00:00, ?B/s]

topk.py: 0.00B [00:00, ?B/s]

_topk_forward.py: 0.00B [00:00, ?B/s]

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/165 [00:00<?, ?B/s]

## App
- to run fullscreen click on the link below (after * Running on public URL:)
- you can also share this link with others; it will work as long as your notebook is running

In [ ]:
chatbot()

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://9a445b0e0782b74a5c.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
